In [1]:
import os
import sys

sys.path.insert(0, os.path.abspath(".."))

import torch
import torch.nn as nn
from torch.optim import Adam
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from utils import load_configs
from train import train
from models.autoencoder import AutoEncoder
from preprocessing import GaussianFilter

configs = load_configs("../configs.json")

In [2]:
model = AutoEncoder()

optimizer = Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss(reduction="sum")

epochs = 30
batch_size = 1000
n_plots = 64

root = configs["dataset_path"]

download = os.path.exists(os.path.join(root,"MNIST"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

trainset = datasets.MNIST(root=root, download=download, train=True, transform=transforms.ToTensor())
testset = datasets.MNIST(root=root, download=download, train=False, transform=transforms.ToTensor())

trainloader = DataLoader(trainset, shuffle=True, batch_size=batch_size)
testloader = DataLoader(testset, shuffle=False, batch_size=batch_size)

preprocessing = GaussianFilter(1,7,1,3)

train(
    model,
    trainloader,
    testloader,
    epochs,
    optimizer,
    loss_fn,
    device,
    n_plots,
    preprocessing=preprocessing,
    save_dir=".."
)

30/30 epochs: 100%|██████████| 60/60 [00:22<00:00,  2.72it/s, train_reconstruction_loss=76.1, train_mse=0.0116, train_psnr=20.1, train_ssim=0.926, test_reconstruction_loss=87.7, test_mse=0.017, test_psnr=18, test_ssim=0.88]
